In [11]:
import os
import sys
sys.path.append(".")
sys.path.append("..")
import imodels
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from imodels.tree.rf_plus.rf_plus.rf_plus_models import RandomForestPlusRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_auc_score, f1_score, recall_score, precision_score, mean_squared_error, r2_score, average_precision_score
from imodels.tree.rf_plus.feature_importance.rfplus_explainer import *
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
from feature_importance.scripts.simulations_util import *
from scripts.competing_methods_local import *
from rbo_implementation import rbo_dict

##### Debug the differences yielded by AUROC and RBO

In [12]:
X = sample_normal_X(n_train=250, n_test=100, d=10, seed=42)
y, support,beta = linear_model(X=X, beta=1, sigma=None, heritability=0.8, s=5, seed=42, return_support=True)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=100, random_state=0)
est = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
est.fit(X_train, y_train)
# rf_plus_base = RandomForestPlusRegressor(rf_model=est)
# rf_plus_base.fit(X_train, y_train)
rf_plus_base_oob = RandomForestPlusRegressor(rf_model=est, fit_on="oob")
rf_plus_base_oob.fit(X_train, y_train)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    3.0s finished


In [14]:
explainer = shap.TreeExplainer(est)
local_fi_score_train_shap = np.abs(explainer.shap_values(X_train, check_additivity=False))
local_fi_score_test_shap = np.abs(explainer.shap_values(X_test, check_additivity=False))

In [15]:
rf_plus_mdi_train = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="oob")
rf_plus_mdi_test = AloRFPlusMDI(rf_plus_base_oob, evaluate_on="all")
local_fi_score_train_lmdi_plus = np.abs(rf_plus_mdi_train.explain_subtract_intercept(X=X_train, y=y_train))
local_fi_score_test_lmdi_plus = np.abs(rf_plus_mdi_test.explain_subtract_intercept(X=X_test, y=None))

In [16]:
local_fi_score_train_lmdi_plus_avg = np.abs(rf_plus_mdi_train.explain_subtract_intercept(X=X_train, y=y_train, leaf_average=True))
local_fi_score_test_lmdi_plus_avg = np.abs(rf_plus_mdi_test.explain_subtract_intercept(X=X_test, y=None, leaf_average=True))

In [17]:
auroc_shap = []
rbo_lst_09_shap = []
num_captured_shap = []
for i in range(local_fi_score_test_shap.shape[0]):
    fi_data_i = local_fi_score_test_shap[i]
    ground_truth_fi_i = np.abs(X_test)[i]
    ground_truth_fi_i[support == 0] = 0
    dict_predictions = dict(enumerate(fi_data_i))
    dict_ground_truth = dict(enumerate(ground_truth_fi_i))      
    num_signal_features = int(np.sum(support))            
    auroc_shap.append(roc_auc_score(support, fi_data_i))
    rbo_lst_09_shap.append(rbo_dict(dict1=dict_ground_truth, dict2=dict_predictions, p=0.9)[2])
    sorted_indices = np.argsort(-fi_data_i)
    top_indices = sorted_indices[:num_signal_features]
    actual_signal_features = np.sum(support[top_indices])
    num_captured_shap.append(actual_signal_features)

In [18]:
auroc_lmdi_plus= []
rbo_lst_09_lmdi_plus = []
num_captured_lmdi_plus = []
for i in range(local_fi_score_test_lmdi_plus.shape[0]):
    fi_data_i = local_fi_score_test_lmdi_plus[i]
    ground_truth_fi_i = np.abs(X_test)[i]
    ground_truth_fi_i[support == 0] = 0
    dict_predictions = dict(enumerate(fi_data_i))
    dict_ground_truth = dict(enumerate(ground_truth_fi_i))      
    num_signal_features = int(np.sum(support))            
    auroc_lmdi_plus.append(roc_auc_score(support, fi_data_i))
    rbo_lst_09_lmdi_plus.append(rbo_dict(dict1=dict_ground_truth, dict2=dict_predictions, p=0.9)[2])
    sorted_indices = np.argsort(-fi_data_i)
    top_indices = sorted_indices[:num_signal_features]
    actual_signal_features = np.sum(support[top_indices])
    num_captured_lmdi_plus.append(actual_signal_features)

In [19]:
auroc_lmdi_plus_avg= []
rbo_lst_09_lmdi_plus_avg = []
num_captured_lmdi_plus_avg = []
for i in range(local_fi_score_test_lmdi_plus_avg.shape[0]):
    fi_data_i = local_fi_score_test_lmdi_plus_avg[i]
    ground_truth_fi_i = np.abs(X_test)[i]
    ground_truth_fi_i[support == 0] = 0
    dict_predictions = dict(enumerate(fi_data_i))
    dict_ground_truth = dict(enumerate(ground_truth_fi_i))      
    num_signal_features = int(np.sum(support))            
    auroc_lmdi_plus_avg.append(roc_auc_score(support, fi_data_i))
    rbo_lst_09_lmdi_plus_avg.append(rbo_dict(dict1=dict_ground_truth, dict2=dict_predictions, p=0.9)[2])
    sorted_indices = np.argsort(-fi_data_i)
    top_indices = sorted_indices[:num_signal_features]
    actual_signal_features = np.sum(support[top_indices])
    num_captured_lmdi_plus_avg.append(actual_signal_features)

In [20]:
print(np.array(auroc_shap).mean(), np.array(rbo_lst_09_shap).mean(), np.array(num_captured_shap).mean())
print(np.array(auroc_lmdi_plus).mean(), np.array(rbo_lst_09_lmdi_plus).mean(), np.array(num_captured_lmdi_plus).mean())
print(np.array(auroc_lmdi_plus_avg).mean(), np.array(rbo_lst_09_lmdi_plus_avg).mean(), np.array(num_captured_lmdi_plus_avg).mean())

0.9764 0.8816704110163137 4.74
0.7968000000000001 0.8908872110163135 3.89
0.7203999999999998 0.8382172010163136 3.56


In [ ]:
auroc_shap[5], rbo_lst_09_shap[5]

In [ ]:
auroc_lmdi_plus[5], rbo_lst_09_lmdi_plus[5]

In [ ]:
ground_truth_fi_i = np.abs(X_test)[5]
ground_truth_fi_i[support == 0] = 0
ground_truth_fi_i

In [ ]:
p = 0.9

In [ ]:
dict_predictions = dict(enumerate(local_fi_score_test_shap[5]))
dict_ground_truth = dict(enumerate(ground_truth_fi_i))
rbo_dict(dict1=dict_ground_truth, dict2=dict_predictions, p=p, verbose=True)[2]

In [ ]:
local_fi_score_test_lmdi_plus[5]

In [ ]:
dict_predictions = dict(enumerate(local_fi_score_test_lmdi_plus[5]))
dict_ground_truth = dict(enumerate(ground_truth_fi_i))
rbo_dict(dict1=dict_ground_truth, dict2=dict_predictions, p=p, verbose=True)[2]

In [ ]:
temp = np.array([0.30647695  , 0.17410994, 0.816055, 0.17842848, 0.10012125,
       0.26276102, 0.26671546, 0.28039733, 0.23719995, 0.25739759])

In [ ]:
dict_predictions = dict(enumerate(temp))#local_fi_score_test_lmdi_plus[5]))
dict_ground_truth = dict(enumerate(ground_truth_fi_i))
rbo_dict(dict1=dict_ground_truth, dict2=dict_predictions, p=p, verbose=True)[2]

##### Debug two group setting with intercept

In [ ]:
X = sample_normal_X_subgroups(n = 500, d=10, mean= [[0]*10,[0]*5+[0]*5], scale =[[1]*10,[1]*10])
temp = linear_model(X, beta=1, sigma=None, heritability=0.6, s=5, return_support=True)
y = temp[0]
support = temp[1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
linear_model(X, beta=1, sigma=None, heritability=0.2, s=5, return_support=True)[1]

In [ ]:
rf = RandomForestRegressor(n_estimators=100, min_samples_leaf=5, max_features=0.33, random_state=42)
rf.fit(X_train, y_train)

In [ ]:
rf_plus_base = RandomForestPlusRegressor(rf_model=rf)
rf_plus_base.fit(X_train, y_train)

# rf_plus_base_oob = RandomForestPlusRegressor(rf_model=rf, fit_on="oob")
# rf_plus_base_oob.fit(X_train, y_train)

# rf_plus_base_inbag = RandomForestPlusRegressor(rf_model=rf, include_raw=False, fit_on="inbag", prediction_model=Ridge(alpha=1e-6))
# rf_plus_base_inbag.fit(X_train, y_train)

In [ ]:
np.random.seed(42)
if X_train.shape[0] > 100:
    indices_train = np.random.choice(X_train.shape[0], 100, replace=False)
    X_train_subset = X_train[indices_train]
    y_train_subset = y_train[indices_train]
else:
    indices_train = np.arange(X_train.shape[0])
    X_train_subset = X_train
    y_train_subset = y_train

if X_test.shape[0] > 100:
    indices_test = np.random.choice(X_test.shape[0], 100, replace=False)
    X_test_subset = X_test[indices_test]
    y_test_subset = y_test[indices_test]
else:
    indices_test = np.arange(X_test.shape[0])
    X_test_subset = X_test
    y_test_subset = y_test

In [ ]:
indices_train_0 = np.where(X_train_subset[:, -1] == 0)[0]
indices_test_0 = np.where(X_test_subset[:, -1] == 0)[0]

In [ ]:
indices_train_1 = np.where(X_train_subset[:, -1] == 1)[0]
indices_test_1 = np.where(X_test_subset[:, -1] == 1)[0]

In [ ]:
local_fi_score_train, local_fi_score_train_subset, local_fi_score_test, local_fi_score_test_subset = tree_shap_evaluation_RF(X_train=X_train, y_train=y_train, X_train_subset = X_train_subset, y_train_subset=y_train_subset,X_test=X_test, y_test=y_test, X_test_subset=X_test_subset, y_test_subset=y_test_subset,fit=rf, mode="absolute")

In [ ]:
new_support_train = np.abs(X_train_subset)
new_support_test = np.abs(X_test)
new_support_train[:, -5:] = 0
new_support_test[:, -5:] = 0

In [ ]:
data = local_fi_score_train_subset
auroc = []
auprc = []
for i in range(data.shape[0]):
        auroc.append(roc_auc_score(support, data[i]))
        auprc.append(average_precision_score(support, data[i]))
print("Treeshap Trainsubset")
print(np.array(auroc).mean())
print(np.array(auprc).mean())

In [ ]:
data = local_fi_score_test 
auroc = []
auprc = []
for i in range(data.shape[0]):
        auroc.append(roc_auc_score(support, data[i]))
        auprc.append(average_precision_score(support, data[i]))
print("Treeshap Test")
print(np.array(auroc).mean())
print(np.array(auprc).mean())

In [ ]:
def rank_biased_overlap(list1, list2, p=0.9):
    """
    Compute the Rank-Biased Overlap (RBO) between two ranked lists.

    Parameters:
    - list1: numpy array or list of the first ranked list
    - list2: numpy array or list of the second ranked list
    - p: the discount factor (default is 0.9, which is commonly used)

    Returns:
    - rbo: the Rank-Biased Overlap score
    """

    # Convert lists to numpy arrays if they're not already
    list1 = np.asarray(list1)
    list2 = np.asarray(list2)

    # Get the indices that would sort the arrays in descending order
    sorted_indices1 = np.argsort(-list1)
    sorted_indices2 = np.argsort(-list2)

    # Rank lists based on sorted indices
    ranked_list1 = sorted_indices1
    ranked_list2 = sorted_indices2

    # Initialize the overlap
    overlap = 0.0
    min_len = min(len(ranked_list1), len(ranked_list2))
    
    # Compute the RBO
    for i in range(min_len):
        # Calculate the overlap at rank i
        rank_i_overlap = len(set(ranked_list1[:i+1]) & set(ranked_list2[:i+1]))
        
        # Add the discounted overlap to the total
        overlap += (rank_i_overlap / (i + 1)) * (p ** (i + 1))
    
    # Normalize the score
    normalization = (1 - p) / (1 - p ** (min_len + 1))
    rbo = overlap * normalization
    
    return rbo

In [ ]:
data = local_fi_score_train_subset
rbo = []
for i in range(data.shape[0]):
        rbo.append(rank_biased_overlap(new_support_train[i], data[i]))
print("Treeshap Trainsubset")
print(np.array(rbo).mean())

In [ ]:
data = local_fi_score_test
rbo = []
for i in range(data.shape[0]):
        rbo.append(rank_biased_overlap(new_support_test[i], data[i]))
print("Treeshap Trainsubset")
print(np.array(rbo).mean())

In [ ]:
rf_plus_mdi_train = AloRFPlusMDI(rf_plus_base, evaluate_on="oob")
rf_plus_mdi_test = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
local_fi_score_train = np.abs(rf_plus_mdi_train.explain_subtract_intercept(X=X_train, y=y_train))
local_fi_score_test = np.abs(rf_plus_mdi_test.explain_subtract_intercept(X=X_test, y=None))
local_fi_score_test_subset = np.abs(rf_plus_mdi_test.explain_subtract_intercept(X=X_test_subset, y=None))
local_fi_score_train_subset = local_fi_score_train[indices_train]

In [ ]:
data = local_fi_score_train_subset 
auroc = []
auprc = []
for i in range(data.shape[0]):
        auroc.append(roc_auc_score(support, data[i]))
        auprc.append(average_precision_score(support, data[i]))
print(np.array(auroc).mean())
print(np.array(auprc).mean())

In [ ]:
data = local_fi_score_test 
auroc = []
auprc = []
for i in range(data.shape[0]):
        auroc.append(roc_auc_score(support, data[i]))
        auprc.append(average_precision_score(support, data[i]))
print(np.array(auroc).mean())
print(np.array(auprc).mean())

In [ ]:
new_support_train = np.abs(X_train_subset)
new_support_test = np.abs(X_test)
new_support_train[:, -5:] = 0
new_support_test[:, -5:] = 0

In [ ]:
data = local_fi_score_train_subset
rbo = []
for i in range(data.shape[0]):
        rbo.append(rank_biased_overlap(new_support_train[i], data[i]))
print("Treeshap Trainsubset")
print(np.array(rbo).mean())

In [ ]:
data = local_fi_score_test
rbo_lst = []
for i in range(data.shape[0]):
        rbo_lst.append(rbo.RankingSimilarity(new_support_test[i], data[i]).rbo())#rbo.append(rank_biased_overlap(new_support_test[i], data[i]))
print("Treeshap Trainsubset")
print(np.array(rbo).mean())

In [ ]:
new_support_test[0]

In [ ]:
data[0]

In [ ]:
rbo.RankingSimilarity(S, T).rbo()

In [ ]:
rf_plus_mdi_train = AloRFPlusMDI(rf_plus_base, evaluate_on="oob")
rf_plus_mdi_test = AloRFPlusMDI(rf_plus_base, evaluate_on="all")
local_fi_score_train = np.abs(rf_plus_mdi_train.explain(X=X_train, y=y_train)[1])
local_fi_score_test = np.abs(rf_plus_mdi_test.explain(X=X_test, y=None)[1])
local_fi_score_test_subset = np.abs(rf_plus_mdi_test.explain(X=X_test_subset, y=None)[1])
local_fi_score_train_subset = local_fi_score_train[indices_train]

In [ ]:
data = local_fi_score_train_subset 
auroc = []
auprc = []
for i in range(data.shape[0]):
        auroc.append(roc_auc_score(support, data[i]))
        auprc.append(average_precision_score(support, data[i]))
print(np.array(auroc).mean())
print(np.array(auprc).mean())

In [ ]:
data = local_fi_score_test 
auroc = []
auprc = []
for i in range(data.shape[0]):
        auroc.append(roc_auc_score(support, data[i]))
        auprc.append(average_precision_score(support, data[i]))
print(np.array(auroc).mean())
print(np.array(auprc).mean())